## This notebook is for testing scraping. I'll be using the [Gwent:The Witcher Card Game website](https://www.playgwent.com/) to scrape the information I need.
## Tools: 
* Selenium
* BeautifulSoup
* Pandas
* Jupyter Notebook.

In [72]:
#selenium with chrome
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

driver = webdriver.Chrome('C:/Users/User/Downloads/chromedriver_win32/chromedriver.exe')

def get_stats(player):
    url = 'https://www.playgwent.com/es/profile/' + player
    driver.get(url)
    driver.implicitly_wait(5)
    #driver.refresh()
    driver.implicitly_wait(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    current_stats = soup.find('table', {'class': 'c-statistics-table current-ranked'})
    df = pd.read_html(str(current_stats))
    df[0].rename(columns = {'Clasificación de la temporada actual':'Stats', 'Clasificación de la temporada actual.1':'value'}, inplace = True)
    df=df[0].drop(['Clasificación de la temporada actual.2'], axis=1)
    df.value = df.value.str.replace('partidas', '')
    df = df.iloc[0:9,:]
    df1 = df.set_index('Stats').T
    df1['player'] = player
    return df1

C:\Users\User\AppData\Local\Temp/ipykernel_18348/1676004483.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/User/Downloads/chromedriver_win32/chromedriver.exe')


In [26]:
##Test with one name that I found
df1 = get_stats('GrandKazuzu')

In [38]:
## Find the better 20 players of the current season
url = 'https://masters.playgwent.com/en/rankings/masters-4/may-season'
driver.get(url)
driver.implicitly_wait(5)
soup = BeautifulSoup(driver.page_source, 'html.parser')
tables = soup.find_all('table')

In [39]:
#Ranking
ranking = soup.find_all('div', {'class': 'c-ranking-table__tr'})[0].find('div', {'class': 'td-number'}).text
#Nick
nick = soup.find_all('div', {'class': 'c-ranking-table__tr'})[0].find('div', {'class': 'td-nick'}).text
#matches
matches = soup.find_all('div', {'class': 'c-ranking-table__tr'})[0].find('div', {'class': 'td-matches'}).text
#mmr
mmr = soup.find_all('div', {'class': 'c-ranking-table__tr'})[0].find('div', {'class': 'td-mmr'}).text

In [40]:
print(ranking, nick, matches, mmr)

1 GrandKazuzu 411 matches       10,377
  


In [41]:
info = soup.find_all('div', {'class': 'c-ranking-table__tr'})
print(len(info))

40


In [42]:
df_better = pd.DataFrame(columns = ['ranking', 'nick', 'matches', 'mmr'])

In [43]:
for i in range(20):
    ranking = info[i].find('div', {'class': 'td-number'}).text
    nick = info[i].find('div', {'class': 'td-nick'}).text
    matches = info[i].find('div', {'class': 'td-matches'}).text
    mmr = info[i].find('div', {'class': 'td-mmr'}).text
    df_better.loc[i] = [ranking, nick, matches, mmr]


In [89]:
df_by_player = pd.DataFrame(columns = ['value','Total', 'Victorias', 'Derrotas', 'Draws', 'Reinos del Norte',
       'Sindicato', 'Monstruos', "Scoia'tael", 'Nilfgaard', 'player'] )
df_by_player.set_index('value', inplace=True)

In [99]:
for i in df_better.nick:
    df_by_player = pd.concat([df_by_player, get_stats(i)])